函数就像机械加工，着重于处理过程；类则关注数据本身，使其“活”过来。  
类存在两种关系：继承(inhertiance)自某个族类，组合(composition)了哪些部件。   
建议将类与业务逻辑分离。逻辑代表业务处理过程，其通常被设计为无状态，用上下文传递数据。业务逻辑讲究效率和并发性，以及相对固定的执行过程。逻辑适合以函数分段实现，按特定顺序组装，并以模块存储。这些特点都不适合以数据为核心，且注重家族遗传，用多实例体现个性的类来实现。

In [1]:
def test():
    class X:
        data =100
        def get(self):
            return self.data


In [2]:
import dis
dis.dis(test)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object X at 0x111a47540, file "<ipython-input-1-47c898501eae>", line 2>)
              4 LOAD_CONST               2 ('X')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('X')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (X)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


先创建了X函数，其内容是属性设置和方法创建。随后该函数被当作参数传递给buildins.\_\_build_class__调用，这里其实就是元类在执行。

In [4]:
dis.dis(test.__code__.co_consts[1])

  2           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('test.<locals>.X')
              6 STORE_NAME               2 (__qualname__)

  3           8 LOAD_CONST               1 (100)
             10 STORE_NAME               3 (data)

  4          12 LOAD_CONST               2 (<code object get at 0x111a47c90, file "<ipython-input-1-47c898501eae>", line 4>)
             14 LOAD_CONST               3 ('test.<locals>.X.get')
             16 MAKE_FUNCTION            0
             18 STORE_NAME               4 (get)
             20 LOAD_CONST               4 (None)
             22 RETURN_VALUE


- 类型有自己的名字空间，存储当前类定义的属性和方法，这其中并不包括所继承的祖先成员，其以引用关联祖先类型，无须复制本地
- 实例会存储所有继承层次的实例字段，这些都属于其私有数据
- 类型的名字空间返回mappingproxy只读视图，不允许直接修改；实例的名字空间就是普通字典，可直接修改
- dir函数搜索所有可访问成员的名字，vars直接返回\_\_dict__属性



In [5]:
def enclosing():
    a = 'enclosing.a'
    class A:
        a = 'A.a'
        def test(self):
            print('E.a=',a)
        print('A.a=',a)
        print('A.locals=',locals())
    A().test()
enclosing()

A.a= A.a
A.locals= {'__module__': '__main__', '__qualname__': 'enclosing.<locals>.A', 'a': 'A.a', 'test': <function enclosing.<locals>.A.test at 0x111a4fd90>}
E.a= enclosing.a


双下划线开头的私有字段会被重命名，这也导致其继承类型无法访问重命名后的基类成员，这也是单下划线私有字段的一个原因

- 属性（property）机制就是将读、写和删除操作映射到指定的方法调用上，从而实现操作控制
- 属性可实现延迟初始化，无须提前准备数据。且属性方法调用未必会绑定字段，可按需要从任意数据源获取
- 尽管属性保存于类型的名字空间，但其优先级高于同名实例字段

In [8]:
class X:
    def __init__(self, name):
        self.__name = name
    @property
    def name(self):
        return self.__name
    @name.setter
    def name(self, value):
        self.__name = value

In [9]:
vars(X)

mappingproxy({'__dict__': <attribute '__dict__' of 'X' objects>,
              '__doc__': None,
              '__init__': <function __main__.X.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'X' objects>,
              'name': <property at 0x111a7a188>})

In [11]:
vars(X('haha'))

{'_X__name': 'haha'}

In [12]:
class X:
    def test(self):
        pass
o = X()

In [16]:
o.test

<bound method X.test of <__main__.X object at 0x111aecc50>>

In [17]:
X.test

<function __main__.X.test>

同一个方法，仅因为引用方式不同，结果就有绑定方法和函数的区别，这其中是描述符机制在起作用。它会将实例附加到方法的\_\_self__属性，解释器执行时就会隐式传入self参数。

In [18]:
o.test.__self__

In [19]:
X.test.__self__

AttributeError: 'function' object has no attribute '__self__'

类型方法用来维护类型状态，面向族群提供服务接口，其实现方式与实例方法完全相同，且存储类型引用的字段名也还是\_\_self__   
静态方法总是函数

- 封装讲究结构复用，逻辑内敛，以固定接口对外提供服务。其遵循单一职责，规定每个类仅有一个引发变化的原因。多于一个的耦合设计会导致脆弱类型，任何职责的变更可能引发连带事故
- 继承并非复制原有类型，而是一种增量进化。其对应开闭原则，对修改封闭，对扩展开放。此外不应违背里氏替换原则，也就是多态特征，所有继承子类应该能直接用于引用父类的场合
- 我们也习惯于将复杂类型的公用部分剥离出来，形成稳固的抽象基类。其他引发变化的相似因素被分离成多个子类，以确保单一职责得到遵守，并能相互替换

In [3]:
class A: pass
class B(A):pass
class C(B):pass

In [2]:
A.__class__

type

In [5]:
A.__subclasses__()

[__main__.B]

In [6]:
B.__base__

__main__.A

In [7]:
C.__base__

__main__.B

In [11]:
C.__bases__

(__main__.B,)

In [8]:
class D:
    def m(self): print('D.m')
    def do(self): self.m()
import dis
dis.dis(D.do)

  3           0 LOAD_FAST                0 (self)
              2 LOAD_ATTR                0 (m)
              4 CALL_FUNCTION            0
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


多继承从优点上说提供了一种混入机制，让既有体系轻松扩展出其他体系的功能。但它会导致严重的混乱，新语言大多会避开多继承，以接口实现。  
\_\_base__仅返回单个基类的只读属性，\_\_bases__按继承顺序返回全部基类。  
MRO（method resolution order）即搜索的基本规则。

In [12]:
C.__mro__

(__main__.C, __main__.B, __main__.A, object)

In [20]:
class A:
    def test(self):
        print('A')
class B(A):
    def test(self):
        print('B')
class C(B):
    def test(self):
        print('C')

In [22]:
super(C,C).test

<function __main__.B.test>

In [23]:
super(B,C).test

<function __main__.A.test>

In [24]:
super(A,C).test

AttributeError: 'super' object has no attribute 'test'

In [25]:
super(C(),C).test

TypeError: super() argument 1 must be type, not C

In [26]:
super(C,C()).test

<bound method B.test of <__main__.C object at 0x1037f4be0>>

super的第一个参数用来指定起点，第二个参数用来提供mro列表。函数总是返回起点位置后一类型。

抽象类表示部分完成，且不能被实例化的类。从抽象类继承，必须实现所有层级未被实现的抽象方法，否则仍然是抽象类无法创建实例。

In [27]:
from abc import ABCMeta, abstractmethod
class Store(metaclass=ABCMeta):
    def __init__(self, name):
        self.name = name
    @abstractmethod
    def dump(self, data):
        pass
    
    @classmethod
    @abstractmethod
    def hello(cls):
        pass

In [28]:
s = Store('ss')

TypeError: Can't instantiate abstract class Store with abstract methods dump, hello

In [29]:
class X:
    pass
o = X()  # 预先创建实例

In [30]:
X.a = lambda self: print('instance method:',self) # 动态添加方法
X.b = classmethod(lambda cls: print('class method:', cls))
X.c = staticmethod(lambda: print('static method'))

In [31]:
o.a()

instance method: <__main__.X object at 0x10377a780>


In [32]:
o.b()

class method: <class '__main__.X'>


In [33]:
o.c()

static method


SimpleNamespace简单继承自object，用于替代"class X:pass"之类的语句。  
不同于nametuple创建结构化类型，其可直接创建实例。

In [38]:
import types
o = types.SimpleNamespace(a='sds')

In [39]:
o.__dict__

{'a': 'sds'}

名字空间字典带来便利的同时，也会造成内存和性能问题。对于需要创建海量实例的类型，可以使用\_\_slots__阻止实例创建dict等成员，仅为指定成员分配空间。  
对于有slots设置的类型，解释器在创建类型对象时，直接讲指定成员包装成描述符后静态分配到类型对象尾部。实例字段也不再通过\_\_dict__存储，而是改为直接分配引用内存。成员后续操作都由描述符完成，可间接修改或删除字段内容，但却无法改变描述符本身。  
继承有slots设置的类型，同样需要添加该设置。其可为空，或是新增设置，以指示解释器继续使用特定分配策略。